In [1]:
pip install xlsxwriter

In [2]:
# import the libraries
from textblob import TextBlob
# needed for the library to work in Google Colab
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords

import tweepy
import string


from wordcloud import WordCloud

import pandas as pd
import numpy as np

import re
import json
from IPython.display import HTML

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#connecting to twitter 

#storing a csv file that contains information needed to access Twitter API into some variable. You can directly insert keys and tokens into code.

TwitterAPI=pd.read_csv("TwitterKey.csv")

In [4]:
# Twitter API credentials

# Extract all the tokens from CSV files


APIkey=TwitterAPI['keys'][0]
APIsecretkey=TwitterAPI['keys'][1]
APIaccesstoken=TwitterAPI['keys'][2]
APItokensecret=TwitterAPI['keys'][3]

In [5]:
#create authentication object 

# will take key and secret key
#OAuth requires a user to log into the service it is trying to log into

authenticate=tweepy.OAuthHandler(APIkey,APIsecretkey)


#set the access token and access secret


authenticate.set_access_token(APIaccesstoken,APItokensecret)

#create the API object while passing in the auth information

api=tweepy.API(authenticate,wait_on_rate_limit=True)

In [6]:
#gets the list of all the tweets in the account
tweets= list(tweepy.Cursor(api.user_timeline,screen_name="NYPDHateCrimes",count=10,lang="en",tweet_mode="extended").items())

In [7]:
#extract the all the data from json format and create a dictonary
output = []
for tweet in tweets:
  if 'retweeted_status' in dir(tweet):
    text_tweet=tweet.retweeted_status.full_text
  else:
    text_tweet = tweet.full_text
    
  retweet_count = tweet.retweet_count
  created_at = tweet.created_at
  
  line = {'text' : text_tweet,  'retweet_count' : retweet_count, 'created_at' : created_at}
  output.append(line)

In [8]:
#set the column width of the dataframe
pd.set_option('max_colwidth', 200)

In [9]:
#create a dataframe for the output file
twitter = pd.DataFrame(output)


In [10]:
#cleans the tweets
def CleanTweets(text):
  
  text = re.sub(r'@[A-Za-z0-9]+', '',text)   #removes @mentions
  # removes "#"
  text= re.sub(r'#', '',text)
  # removes hyperlink
  text= re.sub(r'https?:\/\/\S+', '',text)
  #removes retweets followed by spaces
  text = re.sub(r'RT[\s]+', '', text)


  return text

In [11]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
#removes the punctuation
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

In [13]:
#apply the function
twitter['filtered_tweets']=twitter['text'].apply(CleanTweets)

In [14]:
#apply the remove punct function
twitter['filtered_tweets']=twitter['filtered_tweets'].apply(remove_punct)

In [15]:
#to extract the links from the tweets
twitter['text']=twitter['text'].astype(str)
pattern = r'(https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}[-a-zA-Z0-9()@:%_+.~#?&/=]*)'
twitter['links'] = ''
twitter['links']= twitter['text'].str.extract(pattern, expand=True)



In [16]:
#removes emoji from the tweets
def deEmojify(text):
    return text.encode('ascii', 'ignore').decode('ascii')

In [17]:
#apply function demojify
twitter['filtered_tweets']=twitter['filtered_tweets'].apply(deEmojify)

In [18]:
#replacing empty strings or row with NAN as python doesn't recognize empty value
twitter['filtered_tweets'].replace(' ', np.nan, inplace=True)

In [19]:
twitter = twitter.replace('\n',' ', regex=True)

In [20]:
# #to make the links clickable
# def make_clickable(val):
#     return f'<a target="_blank" href="{val}">{val}</a>'

# twitter.style.format({'links': make_clickable})

In [21]:
twitter

,text,retweet_count,created_at,filtered_tweets,links
0,"We are closely monitoring the hostage situation in Colleyville, Texas and we’re praying for a peaceful and safe conclusion. Out of an abundance of caution, the NYPD has deployed additional resour...",168,2022-01-16 03:30:50,We are closely monitoring the hostage situation in Colleyville Texas and were praying for a peaceful and safe conclusion Out of an abundance of caution the NYPD has deployed additional resources ...,NaN
1,"UPDATE: The Port Authority of New York and New Jersey Police Department has arrested Mohamed Hassanain for the January 3 attack on ""Mr. Singh,"" a Sikh taxi driver, at JFK airport. The incident is ...",102,2022-01-15 01:44:04,UPDATE The Port Authority of New York and New Jersey Police Department has arrested Mohamed Hassanain for the January attack on Mr Singh a Sikh taxi driver at JFK airport The incident is being co...,https://t.co/PZb6nuS6jC
2,The Sikh Coalition is proud to continue providing legal assistance to Mr. Singh as the investigation moves forward. We thank the PAPD and office of @QueensDAKatz for their prompt action on this ca...,8,2022-01-15 01:07:16,The Sikh Coalition is proud to continue providing legal assistance to Mr Singh as the investigation moves forward We thank the PAPD and office of for their prompt action on this case as well as ...,NaN
3,"🚨ARREST UPDATE from 10/23/21 incident in which an Asian male was assaulted at 303 5th Ave, Manhattan. Arrested: McNamara, Jordan, 18 New York, NY Charge: Assault 3rd Degree ⚠️Still seeking to iden...",33,2022-01-15 00:33:17,ARREST UPDATE from incident in which an Asian male was assaulted at th Ave Manhattan Arrested McNamara Jordan New York NY Charge Assault rd Degree Still seeking to identify individual on the le...,https://t.co/Foe5u0dbVo
4,"🚨ARREST UPDATE🚨 Hate Crime Task Force Detectives, assisted by Manhattan South Warrants, have arrested the individual wanted for this anti-Asian hate crime. Arrested: Wactor, Ebony 41 Manhattan C...",27,2022-01-13 20:05:41,ARREST UPDATE Hate Crime Task Force Detectives assisted by Manhattan South Warrants have arrested the individual wanted for this antiAsian hate crime Arrested Wactor Ebony Manhattan Charges Ass...,https://t.co/GJPRkKERSp
...,...,...,...,...,...
488,APPREHENDED: Andrew Oshesky has been arrested by the @NYPDHateCrimes for destroying &amp; desecrating (urinating upon) religious statues in front of a Brooklyn Roman Catholic Church on 12-2-18. Am...,29,2018-12-10 20:59:14,APPREHENDED Andrew Oshesky has been arrested by the for destroying amp desecrating urinating upon religious statues in front of a Brooklyn Roman Catholic Church on Among the charges Criminal Mis...,https://t.co/MS6cIVjsmF
489,UPDATE: Perpetrator arrested for URINATING on and TOPPLING religious statues in #Williamsburg @NYPD90Pct Excellent work and community assistance. https://t.co/x3Z6mQy6Db,12,2018-12-10 20:31:29,UPDATE Perpetrator arrested for URINATING on and TOPPLING religious statues in Williamsburg Excellent work and community assistance,https://t.co/x3Z6mQy6Db
490,NYPD Hate Crime Task Force joins in sharing our condolences on the passing of this American hero. https://t.co/i9iOHScr5Y,2,2018-12-09 05:02:51,NYPD Hate Crime Task Force joins in sharing our condolences on the passing of this American hero,https://t.co/i9iOHScr5Y
491,WANTED for punching a 20 year old female after making ANTI-GAY statement. Confines of the 112 Pct #foresthills https://t.co/9oLhSuXdsX,27,2018-12-08 18:58:29,WANTED for punching a year old female after making ANTIGAY statement Confines of the Pct foresthills,https://t.co/9oLhSuXdsX


In [22]:


# Creating an excel file

writer = pd.ExcelWriter("nypd_tweet.xlsx", engine='xlsxwriter')

twitter.to_excel(writer, index=False)

workbook  = writer.book

#worksheet.write_url(0,4,'')


writer.save()